Most important Hypothesis for me in this project is:

1. Hypothesis 03. The more client has ability to pay off(istead of making default micro finance), they will choose 5 weeks credit ease product.
-> 10 weeks credit ease clients make highest miss-instalments rate, 7 weeks credit ease clients make highest default rate.
-> 7, 10 weeks credit ease clients borrow more amount of money, less payback on time so it's risk for company.
# how about set up feature transpormation [ Weight of Evidence (WoE) Encoding ] ?

2. Hypothesis 05. Risk of default micro finance is different by branch as their lifestlye, quality of life, average salary might be different.
-> Engineer branch has higher rate to make defaulty. Depends on geographic, feature of city, risk of default rate can be impacted.
# By using [ Branch × BusinessType Target Encoding ] we could predict high risk client ?

3. Hypothesis 06. Frequent client has high risk to make default micro finance.
-> Client those who use product 3-4 times has highest risk to make defaulty while 1-2, 5-6 times clients have lower risk.
# How about set up [ Loan Cycle Binning ] so that AI can give high risk point according to cycle ?


In [3]:
# Import necessary libraries

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt 
import seaborn as sns

pd.set_option("display.max_columns", 100)

# Load the main dataset 
df = pd.read_csv('cleaned_dataset.csv')

# Display the first few rows of the dataset 
df.head()

# BorrowDate:  MM/DD/YYYY (Keep dayfirst=False)
df["BorrowDate"] = pd.to_datetime(df["BorrowDate"], errors="coerce", dayfirst=False)

# problematic columns: Use format='mixed' to handle the mix of DD/MM/YYYY and YYYY-MM-DD
df["loan_repayment_first_payment"] = pd.to_datetime(df["loan_repayment_first_payment"], errors="coerce", dayfirst=True, format='mixed')
df["loan_repayment_last_payment"] = pd.to_datetime(df["loan_repayment_last_payment"], errors="coerce", dayfirst=True, format='mixed')

# ClearDate:  YYYY-MM-DD
df["ClearDate"] = pd.to_datetime(df["ClearDate"], errors="coerce", format="%Y-%m-%d")

# Check the result
df.head()

# More data cleaning 
# A customer cannot make a repayment 100 days BEFORE they borrowed the money.
# We allow a small buffer (-10 days) for potential booking errors, but anything less is garbage.
df['days_to_first'] = (df['loan_repayment_first_payment'] - df['BorrowDate']).dt.days
df = df[(df['days_to_first'] >= -10) | (df['days_to_first'].isna())].copy()

# The data logs the same loan twice. We keep only the first instance.
df = df.drop_duplicates(subset=['CustomerId', 'BorrowDate'], keep='first')

# A loan cannot be cleared BEFORE it was borrowed.
df = df[df['ClearDate'] >= df['BorrowDate']].copy()

# A business established in 2024 cannot borrow money in 2023.
df = df[df['YearEstablished'] <= df['BorrowDate'].dt.year].copy()

# Cleanup to remove Product types (sme - 3 months, sme - 4 months and 12 weeks credit ease) fron the dataset
df = df[~df['Product'].isin(['sme - 3 months', 'sme - 4 months', '12 weeks credit ease'])]
df.info()

df.shape

<class 'pandas.core.frame.DataFrame'>
Index: 1565 entries, 0 to 3469
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   LoanId                        1565 non-null   int64         
 1   AmountDisbursed               1565 non-null   int64         
 2   Interest                      1565 non-null   int64         
 3   LoanBalance                   1565 non-null   float64       
 4   BorrowDate                    1565 non-null   datetime64[ns]
 5   Product                       1565 non-null   object        
 6   Branch                        1565 non-null   object        
 7   ClearDate                     1565 non-null   datetime64[ns]
 8   CustomerId                    1565 non-null   int64         
 9   Gender                        1565 non-null   object        
 10  LoanLimit                     1565 non-null   float64       
 11  CreditScore                   1565 

(1565, 22)

# Feature engineering
1. Feature Selection 
2. Encoding Categorical data
3. Handling Missing values
4. Feature Transformation
5. Balancing datasets

# Feature Selection
Our dataset is already clean, so only drop Identifiers (LoanId, CustomerId)

1. Remove Unique Identifiers: Drop LoanId and CustomerId - no predictive power in actual ML

In [ ]:
# Code to remove LoanId and CustomerId from the datasets
columns_to_drop = ['LoanId', 'CustomerId']
df = df.drop(columns=columns_to_drop)

# Encoding Categorical data

1. Gender & Branch: Use Binary Encoding (0 or 1) - as they only have two categories.

2. Product type :  Use One-Hot Encoding. Since there are only 3 products, this won't create too many columns.

3. Industry & BusinessType: Use Frequency Encoding - replacing the name with its percentage in the dataset.

In [ ]:
#1. Gender & Branch: Use Binary Encoding (0 or 1) - as they only have two categories.
## Binary Encoding for Gener / Branch
df['Gender'] = df['Gender'].map({'male': 1, 'female': 0})
df['Branch'] = df['Branch'].map({'engineer': 1, 'molo': 0})

#2. Product type :  Use One-Hot Encoding. Since there are only 3 products, this won't create too many columns.
## prefix='Product' adds a label to the new columns for better organization
product_dummies = pd.get_dummies(df['Product'], prefix='Product')
df = pd.concat([df, product_dummies], axis=1) # Join the new columns back to the main dataframe
df = df.drop(columns=['Product']) # Drop the original 'Product' column (since it's now encoded)

#3. Industry & BusinessType: Use Frequency Encoding - replacing the name with its percentage in the dataset.
## This replaces the name with the percentage (0.0 to 1.0) of its occurrence
industry_freq = df['Industry'].value_counts(normalize=True)
df['Industry'] = df['Industry'].map(industry_freq)

## Frequency Encoding for BusinessType
business_type_freq = df['BusinessType'].value_counts(normalize=True)
df['BusinessType'] = df['BusinessType'].map(business_type_freq)

# Handling Missing Values
Dataset is already clean and don't have missing values.

1. Convert dates to avoid the error. - James already did in EDA

In [ ]:
df['days_to_first'] = (df['loan_repayment_first_payment'] - df['BorrowDate']).dt.days
df = df[(df['days_to_first'] >= -10) | (df['days_to_first'].isna())].copy()

# Feature Transformation

1. Log Transformation for AmountDisbursed, CreditScore, Loanlimit as those are highly skewed

2. Loan Cycle Binning - According to EDA (Loan Frequency, clients who are in cycle 3-4 have high risk to make feault, while others have lower risk to make defaulty)

3. Scaling (Standardization) - To ensure that a large number like LoanLimit (e.g., 10,000) doesn't overpower a small number like BusinessAge (e.g., 5), we scale all numerical inputs.


In [ ]:
#1. Log Transformation for AmountDisbursed, CreditScore, Loanlimit as those are highly skewed
# Log transform highly skewed variables
skewed_cols = ['AmountDisbursed', 'CreditScore', 'LoanLimit', 'Interest']
for col in skewed_cols:
    df[f'log_{col}'] = np.log1p(df[col])

    

# 2. Loan Cycle Binning - According to EDA (Loan Frequency, clients who are in cycle 3-4 have high risk to make feault, while others have lower risk to make defaulty)
# A. Loan Cycle Binning (Your finding: 3-4 is high risk, 5-6 is lower)
def bin_loan_cycle(count):
    if count <= 2: return 'Low_Freq'
    if 3 <= count <= 4: return 'High_Risk_Cycle'
    if 5 <= count <= 6: return 'Stable_Cycle'
    return 'Veteran'

# Assuming 'LoanCount' is your column for number of loans per client
df['Cycle_Group'] = df['loan_repayment_txn_count'].map(bin_loan_cycle) # or appropriate column

# B. Transaction Count Binning (Your finding: 0-4 highest, 50+ increasing)
df['Txn_Group'] = pd.cut(df['loan_repayment_txn_count'], 
                         bins=[-1, 4, 49, 1000], 
                         labels=['Highest_Risk_Txn', 'Mild_Risk_Txn', 'Increased_Risk_Txn'])

# C. Missed Installment Binning (Your finding: 0 is high, 1-5 lower, 6 highest)
def bin_missed(count):
    if count == 0: return 'No_Miss_Unexpected_Risk'
    if 1 <= count <= 5: return 'Typical_Delay'
    return 'Critical_Default'

df['Missed_Group'] = df['MissedInstalments'].map(bin_missed)



#3. Scaling (Standardization) - To ensure that a large number like LoanLimit (e.g., 10,000) doesn't overpower a small number like BusinessAge (e.g., 5), we scale all numerical inputs.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
num_features = ['BusinessAge', 'log_AmountDisbursed', 'log_CreditScore', 'Industry_Freq']
df[num_features] = scaler.fit_transform(df[num_features])



# 4. Interaction Features - Combining Product type with Credit Score bins to capture nuanced risk profiles.
# Discretize CreditScore into 5 bins
df['CreditScore_Bin'] = pd.qcut(df['CreditScore'], q=5, labels=False)

# Create interaction feature between Product type and Credit Score bin
df['Prod_Credit_Combo'] = df['Product'].astype(str) + "_" + df['CreditScore_Bin'].astype(str)

# Calculate mean default rate for each combination
mean_default = df.groupby('Prod_Credit_Combo')['is_default'].transform('mean')
df['Prod_Credit_DefaultRate'] = mean_default


# 5. Branch × BusinessType Target Encoding
# Create a combined feature of Branch and BusinessType
df['Branch_Biz_Combo'] = df['Branch'].astype(str) + "_" + df['BusinessType'].astype(str)

# Calculate the mean default rate for each combination
# and map it back to the dataframe
branch_biz_risk = df.groupby('Branch_Biz_Combo')['is_default'].transform('mean')
df['Branch_Biz_Risk_Score'] = branch_biz_risk

# Balancing datasets -> Using SMOTE to balance the data when we train the AI

The initial 50/50 split was based on anyone missing an installment. However, our refined business definition of default—unpaid balance past the clear date—shows that only 8.5% of our clients are in actual default. Because this is an imbalanced dataset, we need to use SMOTE or balanced class weights to ensure the model doesn't overlook the minority group.
